## Imports

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from collections import Counter

import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import seaborn as sns
import numpy as np

import imblearn
import optuna
import torch
import json
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Load CLS-tokens and map 'incomplete-classes' to their respective full classes

In [6]:
X = torch.load('/home/evan/D1/project/code/sorted_cls_tokens_features.pt', map_location=device)
#X = torch.load('/home/evan/D1/project/code/stretched_cls_tokens.pt', map_location=device)
#X = torch.load('/home/evan/D1/project/code/reflected_cls_tokens.pt', map_location=device)

y = np.load('/home/evan/D1/project/code/sorted_cls_tokens_labels.npy')
frame_counts = np.load('/home/evan/D1/project/code/frame_counts.npy')


class_mapping = {0:0, 1: 1, 2: 2, 3: 1, 4: 2}

for i, label in enumerate(y):
    y[i] = class_mapping[label]
print('Done')

## Split into train, val on games

In [7]:
# Calculate cumulative start indices of each video in the concatenated tensor
cumulative_starts = np.insert(np.cumsum(frame_counts), 0, 0)[:-1]


split_ratio = (0.7, 0.15, 0.15)  #train, validation, test

num_videos = len(frame_counts)
num_train_videos = int(num_videos * split_ratio[0])
num_val_videos = int(num_videos * split_ratio[1])

# Ensure total does not exceed the number of videos
num_test_videos = num_videos - num_train_videos - num_val_videos

# Shuffle video indices to split into training, validation, and test sets
video_indices = np.arange(num_videos)
np.random.seed(42)
np.random.shuffle(video_indices)

train_video_indices = video_indices[:num_train_videos]
val_video_indices = video_indices[num_train_videos:num_train_videos + num_val_videos]
test_video_indices = video_indices[num_train_videos + num_val_videos:]

# Initialize lists for indices
train_indices, val_indices, test_indices = [], [], []

# Populate the index lists
for idx in train_video_indices:
    start, end = cumulative_starts[idx], cumulative_starts[idx] + frame_counts[idx]
    train_indices.extend(range(start, end))

for idx in val_video_indices:
    start, end = cumulative_starts[idx], cumulative_starts[idx] + frame_counts[idx]
    val_indices.extend(range(start, end))

for idx in test_video_indices:
    start, end = cumulative_starts[idx], cumulative_starts[idx] + frame_counts[idx]
    test_indices.extend(range(start, end))

# Convert indices to tensors and extract corresponding subsets
train_indices = torch.tensor(train_indices)
val_indices = torch.tensor(val_indices)
test_indices = torch.tensor(test_indices)

X_train, y_train = X[train_indices], y[train_indices]
X_val, y_val = X[val_indices], y[val_indices]
X_test, y_test = X[test_indices], y[test_indices]

## Undersample and create undersampled train, val, test datasets

In [8]:
def undersample_data(X, y, target_counts):
    unique_classes, counts = np.unique(y, return_counts=True)
    undersampled_indices = []
    print(counts)
    

    for cls in unique_classes:
        if cls == 0 or cls == 2 or cls == 1:
            cls_indices = np.where(y == cls)[0]

            undersampled_cls_indices = np.random.choice(cls_indices, target_counts[int(cls)], replace=False)
            undersampled_indices.extend(undersampled_cls_indices)
        else:
            cls_indices = np.where(y == cls)[0]
            
            max_count = len(cls_indices)

            undersampled_cls_indices = np.random.choice(cls_indices, max_count, replace=False)
            undersampled_indices.extend(undersampled_cls_indices)

    np.random.shuffle(undersampled_indices)  # Shuffle to mix classes
    X_undersampled = X[undersampled_indices]
    y_undersampled = y[undersampled_indices]

    return X_undersampled, y_undersampled

np.random.seed(42)  # Set a specific random seed for reproducibility

#X_train, y_train = undersample_data(X_train, y_train, {0: 2750, 1: 2750, 2: 5500})
X_train, y_train = undersample_data(X_train, y_train, {0: 4000, 1: 4000, 2: 4000})


#X_val, y_val = undersample_data(X_val, y_val, {0: 688, 1: 688, 2: 688})
#X_val, y_val = undersample_data(X_val, y_val, {0: 500, 1: 500, 2: 500})
X_val, y_val = undersample_data(X_val, y_val, {0: 1000, 1: 1000, 2: 1000})



#X_test, y_test = undersample_data(X_test, y_test, {0: 688, 1: 688, 2: 688})
#X_test, y_test = undersample_data(X_test, y_test, {0: 500, 1: 500, 2: 500})
X_test, y_test = undersample_data(X_test, y_test, {0: 1000, 1: 1000, 2: 1000})

## Check counts

In [9]:
print(np.unique(y_train, return_counts=True))
print(np.unique(y_val, return_counts=True)) 
print(np.unique(y_test, return_counts=True))

## Move all to device

In [10]:
X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
y_train = torch.tensor(y_train, dtype=torch.long, device=device)

X_val = torch.tensor(X_val, dtype=torch.float32, device=device)
y_val = torch.tensor(y_val, dtype=torch.long, device=device)

X_test = torch.tensor(X_test, dtype=torch.float32, device=device)
y_test = torch.tensor(y_test, dtype=torch.long, device=device)

## Create dataloaders for train, val, test

In [11]:
batch_size = 128

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Set folder-name to save data

In [12]:
folder = '/home/evan/D1/project/code/smaller_model/raw/'

if not os.path.exists(folder):
    os.makedirs(folder, exist_ok=True)

## Create and initalize model

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        
        self.fc1 = nn.Linear(1024, 64)
        self.dropout1 = nn.Dropout(0.4)
        self.bn1 = nn.BatchNorm1d(64)
        
        self.fc2 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(0.3)
        self.bn2 = nn.BatchNorm1d(32)
        
        
        self.fc3 = nn.Linear(32, 16)
        self.dropout3 = nn.Dropout(0.2)
        self.bn3 = nn.BatchNorm1d(16)
        
        
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        
        x = self.fc4(x)
        return x

model = BaseModel()
print(model)


## Check model parameters

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

#model = EnhancedMultiLayerClassifier(1024, 3)
print("Number of trainable parameters:", count_parameters(model))


## L1-regularization

In [15]:
def l1_regularization(model, lambda_l1):
    l1_penalty = torch.tensor(0., device=device)  
    for param in model.parameters():
        l1_penalty += torch.norm(param, 1)
    return lambda_l1 * l1_penalty

## Training loop

In [542]:
config = [
{
'lr': 0.00023189475417053056, 'weight_decay': 0.06013631013820486, 'lambda_l1': 7.530339626757409e-05,
'epochs': 80,
'break_margin': 2,
'loss_function': 'CrossEntropy',  # or 'FocalLoss'
'alpha': 0.9186381075849595,  # Only relevant if using FocalLoss
'gamma': 0.2157540954710035   # Only relevant if using FocalLoss
}]

def run_experiment(config):
    #model = EnhancedMultiLayerClassifier(1024, 3).to(device)
    model = BaseModel().to(device)
    
    if config['loss_function'] == 'CrossEntropy':
        criterion = nn.CrossEntropyLoss().to(device)
        
    elif config['loss_function'] == 'FocalLoss':
        # Assuming FocalLoss is defined elsewhere and compatible with your requirements
        criterion = FocalLoss(alpha=config['alpha'], gamma=config['gamma'], reduction='mean').to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
    epochs = config['epochs']
    break_margin = config['break_margin']
    best_f1 = 0.0
    time_to_break = 0
    best_loss = float('inf')
    train_losses, val_losses = [], []
    output_file_path = os.path.join(folder, 'training_output_base.txt')

    
    with open(output_file_path, 'w') as f:
        for epoch in range(epochs):
            model.train()
            train_loss = 0
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)

                # Calculate L1 regularization penalty to prevent overfitting
                l1_penalty = l1_regularization(model, config['lambda_l1'])

                # Add L1 penalty to the loss
                loss += l1_penalty

                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            train_losses.append(train_loss / len(train_loader))

            model.eval()
            val_loss = 0
            all_preds, all_targets, all_outputs = [], [], []
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)
                    val_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    all_preds.extend(predicted.cpu().numpy())
                    all_targets.extend(y_batch.cpu().numpy())
                    all_outputs.extend(outputs.cpu().numpy())
            val_losses.append(val_loss / len(val_loader))

            #precision_0, recall_0, f1_0, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted', zero_division=0)
            precision_0, recall_0, f1_0, _ = precision_recall_fscore_support(all_targets, all_preds, labels=[2], average='macro', zero_division=0)
            accuracy = accuracy_score(all_targets, all_preds)
            
            
            output_str = f'Epoch {epoch+1}: Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Precision: {precision_0:.4f}, Recall: {recall_0:.4f}, F1: {f1_0:.4f}, Accuracy: {accuracy:.4f}\n'
            #f.write(output_str)
            print(output_str, end='')
            
            
            # Save the model if the f1 of the current epoch is the best
            if f1_0 > best_f1:
                best_f1 = f1_0
                best_epoch = epoch
                best_model_state_dict = model.state_dict()
                best_all_targets = all_targets
                best_all_preds = all_preds
                # Define path for saving the model
                best_model_path = os.path.join(folder, 'best_model_for_class_test.pt')
                
            if val_loss < best_loss:
                best_loss = val_loss
                time_to_break = 0
            else:
                time_to_break += 1
                if time_to_break == break_margin:
                    print('Break margin hit')
                    break

    
    return best_model_state_dict, best_all_targets, best_all_preds

for config_index, config_ in enumerate(config):
    print(f'Running configuration {config_index + 1}/{len(config)}')
    best_model_state_dict, all_targets, all_preds = run_experiment(config_)
model.load_state_dict(best_model_state_dict)

## Check entropy/mutual information in features

In [69]:
from sklearn.feature_selection import mutual_info_classif

mi_scores = mutual_info_classif(X_val.cpu(), y_val)


# Calculate the average mutual information per feature
average_mi = np.mean(mi_scores)
print("Average Mutual Information per feature:", average_mi)

plt.bar(range(len(mi_scores)), mi_scores, edgecolor='none')
plt.xlabel('Features')
plt.ylabel('Mutual Information Score')
plt.title('MI Scores for Zero Padded Frame Features')
#plt.savefig("padded_mutual_information_feature.pdf", format="pdf", bbox_inches="tight")
plt.show()

## Optuna optimalization

In [36]:
import logging
import sys
import time

SEED = 13
torch.manual_seed(SEED)


def objective(trial):
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0, 0.1)
    lambda_l1 = trial.suggest_float('lambda_l1', 0, 1e-2)
    #gamma = trial.suggest_float('gamma', 0, 2)
    #alpha = trial.suggest_float('alpha', 0, 1)
    
    model = BaseModel().to(device)

    
    criterion = nn.CrossEntropyLoss().to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    epochs = 40
    best_val_f1 = 0
    epochs_no_improve = 0    
    
    # Now I do. with smaller model!
    early_stop_threshold = 2
    
    train_losses, val_losses = [], []
    #output_file_path = os.path.join(folder, 'base_training_output.txt')

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            # Calculate L1 regularization penalty to prevent overfitting
            l1_penalty = l1_regularization(model, lambda_l1)

            # Add L1 penalty to the loss
            loss += l1_penalty

            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_losses.append(train_loss / len(train_loader))

        model.eval()
        val_loss = 0
        all_preds, all_targets, all_outputs = [], [], []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                all_preds.extend(predicted.cpu().numpy())
                all_targets.extend(y_batch.cpu().numpy())
                all_outputs.extend(outputs.cpu().numpy())
        val_losses.append(val_loss / len(val_loader))

        precision_0, recall_0, f1_0, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted', zero_division=0)
        #precision_0, recall_0, f1_0, _ = precision_recall_fscore_support(all_targets, all_preds, labels=[2], average='macro', zero_division=0)
        accuracy = accuracy_score(all_targets, all_preds)
        
        if f1_0 > best_val_f1:
            best_val_f1 = f1_0
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= early_stop_threshold:
            print("Stopping early due to no improvement")
            break
        trial.report(f1_0, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()
    return f1_0


#optuna.logging.get_logger('optuna').addHandler(logging.StreamHandler(sys.stdout))
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=SEED))







start_time = time.time()
study.optimize(objective, n_trials=150)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Optimization took {elapsed_time:.2f} seconds.")


### Evaluate best model on test test

In [202]:
#model.load_state_dict(torch.load(os.path.join(folder, 'best_model_for_class_test.pt')))
#model.to(device)

model.eval()
test_loss = 0
all_preds = []
all_targets = []


with torch.no_grad(): 
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(y_batch.cpu().numpy())

test_loss /= len(val_loader)
precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted', zero_division=0)
accuracy = accuracy_score(all_targets, all_preds)

test_output_str = f'Test Loss: {test_loss:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Accuracy: {accuracy:.4f}\n'
print(test_output_str)


cm = showConfMatrix(all_targets, all_preds)
showClassWiseAcc(cm)

# Plots and metrics

## Plot imports

In [24]:
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from itertools import cycle
from sklearn.metrics import classification_report

### Metric-classes

In [278]:
def showConfMatrix(all_targets, all_preds):
    conf_matrix = confusion_matrix(all_targets, all_preds)
    # conf_matrix = confusion_matrix(all_preds, all_targets)
    labels = ["background", "tackle-live", "tackle-replay"]
    #labels = ["background", "tackle-live", "tackle-replay", "tackle-live-incomplete", "tackle-replay-incomplete"]


    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    # plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    #plt.savefig(f"{folder}/confusionMatrixSmoteBalance_{epochs_ran}.pdf", format="pdf", bbox_inches="tight")
    plt.show()
    return conf_matrix

In [279]:
def showClassWiseAcc(conf_matrix):
    # Calculate accuracy per class
    class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

    # Prepare accuracy data for writing to file
    accuracy_data = "\n".join([f"Accuracy for class {i}: {class_accuracies[i]:.4f}" for i in range(len(class_accuracies))])

    # Print accuracy per class and write to a file
    print(accuracy_data)  # Print to console

    # Define the filename
    accuracy_file_path = os.path.join(folder, "class_accuracies.txt")

## Confusion Matrix

In [40]:
cm = showConfMatrix(all_targets, all_preds)

## Accuracy per class

In [545]:
cm = showConfMatrix(all_targets, all_preds)
showClassWiseAcc(cm)
labels = ["background", "tackle-live", "tackle-replay"]

print(classification_report(all_targets, all_preds, target_names=labels))
#torch.save(model.state_dict(), f'{folder}/class_2_74_93_82.pt')


## ROC Curve

In [74]:
y_score= np.array(all_outputs)
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = len(labels) 

y_test_one_hot = np.eye(n_classes)[y_val.cpu()]

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_one_hot[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure()
colors = ['blue', 'red', 'green', 'darkorange', 'purple']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(labels[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
print('Receiver operating characteristic for multi-class')
plt.legend(loc="lower right")
plt.savefig(f"{folder}/ROCCurveSmoteBalance_{epochs_ran}.pdf", format="pdf", bbox_inches="tight")
plt.show()

## Multi-Class Precision-Recall Cruve

In [75]:
y_test_bin = label_binarize(y_val.cpu(), classes=range(n_classes))

precision_recall = {}

for i in range(n_classes):
    precision, recall, _ = precision_recall_curve(y_test_bin[:, i], y_score[:, i])
    precision_recall[i] = (precision, recall)

colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])

plt.figure(figsize=(6, 4))

for i, color in zip(range(n_classes), colors):
    precision, recall = precision_recall[i]
    plt.plot(recall, precision, color=color, lw=2, label=f'{labels[i]}')

plt.xlabel('Recall')
plt.ylabel('Precision')
print('Multi-Class Precision-Recall Curve')
plt.legend(loc='best')
plt.savefig(f"{folder}/MultiClassPRCurveSmoteBalance_{epochs_ran}.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Meta Learner

By stacking outputs from 3 models, all with one speciality, we train a meta-model/meta-learner by feeding it all 3 models input and the correct label, so it is able to learn where there are strenghts and weaknesses of the other three models. This is done by stacking the base-models outputs to later use as input data for training the meta-learner.

In [16]:
folder1 = '/home/evan/D1/project/code/smaller_model/raw_training/'

## Generate base-model outputs

In [18]:
model0 = BaseModel().to(device)
model1 = BaseModel().to(device)
model2 = BaseModel().to(device)


    
model0.load_state_dict(torch.load(os.path.join(f'{folder1}', 'class_0/class_0_65_79_71.pt')))
model0.to(device)

model1.load_state_dict(torch.load(os.path.join(f'{folder1}', 'class_1/class_1_74_93_83.pt')))
model1.to(device)

model2.load_state_dict(torch.load(os.path.join(f'{folder1}', 'class_2/class_2_84_90_87.pt')))
model2.to(device)

base_model_outputs = []

with torch.no_grad():
    for X_batch, _ in val_loader:
        X_batch = X_batch.to(device)
        # Store the probabilities, not the class predictions
        probs0 = torch.softmax(model0(X_batch), dim=1)
        probs1 = torch.softmax(model1(X_batch), dim=1)
        probs2 = torch.softmax(model2(X_batch), dim=1)
        
        # Concatenate the model outputs along feature dimension
        model_output = torch.cat((probs0, probs1, probs2), dim=1)
        
        base_model_outputs.append(model_output)

# Stack all batches to form the complete set of base model outputs
base_model_outputs = torch.cat(base_model_outputs, dim=0)


### Meta-Learner class

In [19]:
import torch.nn as nn
import torch.nn.init as init

class MetaModel(nn.Module):
    def __init__(self, input_size, num_classes=3):
        super(MetaModel, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.BatchNorm1d(32),  
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(32, num_classes),
            nn.LogSoftmax(dim=1)
        )
        
        # Apply kaiming initialization to all linear layers
        self.apply(self.initialize_weights)

    def forward(self, x):
        x = self.network(x)
        return x

    def initialize_weights(self, m):
        if isinstance(m, nn.Linear):
            init.kaiming_uniform_(m.weight, nonlinearity='relu')
            if m.bias is not None:
                init.constant_(m.bias, 0)


## Split basemodel-outputs to train, val

In [20]:
y_val = torch.cat([y for _, y in val_loader], dim=0)  # Just to make sure y_val is y_val, extract from val_loader

print(np.unique(y_val.cpu().numpy(), return_counts=True))

X_meta_train, X_meta_val, y_meta_train, y_meta_val = train_test_split(
    base_model_outputs.cpu().numpy(), 
    y_val.cpu().numpy(), 
    test_size=0.2, 
    random_state=42
)

input_size = base_model_outputs.size(1) # Will be 3*num_classes (3 models now) so 9

## Create dataloaders

In [21]:

# Convert numpy arrays back to tensors for training
X_meta_train = torch.tensor(X_meta_train, dtype=torch.float).to(device)
y_meta_train = torch.tensor(y_meta_train, dtype=torch.long).to(device)
X_meta_val = torch.tensor(X_meta_val, dtype=torch.float).to(device)
y_meta_val = torch.tensor(y_meta_val, dtype=torch.long).to(device)

train_meta_dataset = TensorDataset(X_meta_train, y_meta_train)
train_meta_loader = DataLoader(train_meta_dataset, batch_size=64, shuffle=True)

val_meta_dataset = TensorDataset(X_meta_val, y_meta_val)
val_meta_loader = DataLoader(val_meta_dataset, batch_size=64, shuffle=False)


## Optimize meta-learner

In [181]:
import logging
import sys

SEED = 13
torch.manual_seed(SEED)

#criterion = FocalLoss(alpha=1, gamma=2, reduction='mean')

# Convert numpy arrays back to tensors for training
X_meta_train = torch.tensor(X_meta_train, dtype=torch.float).to(device)
y_meta_train = torch.tensor(y_meta_train, dtype=torch.long).to(device)
X_meta_val = torch.tensor(X_meta_val, dtype=torch.float).to(device)
y_meta_val = torch.tensor(y_meta_val, dtype=torch.long).to(device)

train_meta_dataset = TensorDataset(X_meta_train, y_meta_train)
train_meta_loader = DataLoader(train_meta_dataset, batch_size=64, shuffle=True)

val_meta_dataset = TensorDataset(X_meta_val, y_meta_val)
val_meta_loader = DataLoader(val_meta_dataset, batch_size=64, shuffle=False)



def objective(trial):
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0, 0.1)
    lambda_l1 = trial.suggest_float('lambda_l1', 0, 1e-2)
    #gamma = trial.suggest_float('gamma', 0, 2)
    #alpha = trial.suggest_float('alpha', 0, 1)
    
    meta_model = MetaModel(input_size=input_size).to(device) 
    optimizer = torch.optim.Adam(meta_model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    #model0, model1, model2 = get_models()

    
    
    epochs = 400
    best_val_f1 = 0
    epochs_no_improve = 0
    early_stop_threshold = 2
    
    train_losses, val_losses = [], []
    #output_file_path = os.path.join(folder, 'training_output.txt')

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_meta_loader:
            
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            
            outputs = meta_model(X_batch)
            loss = criterion(outputs, y_batch)

            # Calculate L1 regularization penalty to prevent overfitting
            l1_penalty = l1_regularization(model, lambda_l1)

            # Add L1 penalty to the loss
            loss += l1_penalty

            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_losses.append(train_loss / len(train_loader))

        model.eval()
        val_loss = 0
        all_preds, all_targets, all_outputs = [], [], []
        with torch.no_grad():
            for X_batch, y_batch in val_meta_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = meta_model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                all_preds.extend(predicted.cpu().numpy())
                all_targets.extend(y_batch.cpu().numpy())
                all_outputs.extend(outputs.cpu().numpy())
        val_losses.append(val_loss / len(val_loader))

        precision_0, recall_0, f1_0, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted', zero_division=0)
        accuracy = accuracy_score(all_targets, all_preds)
        
        if f1_0 > best_val_f1:
            best_val_f1 = f1_0
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= early_stop_threshold:
            print("Stopping early due to no improvement")
            break
        trial.report(f1_0, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()
    return f1_0

#optuna.logging.get_logger('optuna').addHandler(logging.StreamHandler(sys.stdout))
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=SEED))
study.optimize(objective, n_trials=150)

## Print optuna-stats

In [182]:

# Get the best parameters
best_params = study.best_params

# Print the best parameters
print("Best parameters:", best_params)

# Get the best parameters
best_trial = study.best_trial

# Print the best parameters
print("Best parameters:", best_trial)


### Meta-learner training

Found out that cross entropy gives more stable accuracies across classes.

In [26]:
y_val = torch.cat([y for _, y in val_loader], dim=0)  # Just to make sure y_val is y_val, extract from val_loader

print(np.unique(y_val.cpu().numpy(), return_counts=True))
# Split the data into train, validation, and test sets
X_meta_train, X_meta_temp, y_meta_train, y_meta_temp = train_test_split(
    base_model_outputs.cpu().numpy(), 
    y_val.cpu().numpy(), 
    test_size=0.4, 
    random_state=42
)

X_meta_val, X_meta_test, y_meta_val, y_meta_test = train_test_split(
    X_meta_temp, 
    y_meta_temp, 
    test_size=0.5, 
    random_state=42
)

input_size = base_model_outputs.size(1) # Will be 3*num_classes (3 models now) so 9

config = {
    'lr': 0.007728103291008411, 
    'weight_decay': 0.003503652410143732, 
    'lambda_l1': 0.002984494708891794
}

meta_model = MetaModel(input_size=input_size).to(device) 
optimizer = torch.optim.Adam(meta_model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
lambda_l1 = config['lambda_l1']

criterion = nn.CrossEntropyLoss()

# Convert numpy arrays back to tensors for training
X_meta_train = torch.tensor(X_meta_train, dtype=torch.float).to(device)
y_meta_train = torch.tensor(y_meta_train, dtype=torch.long).to(device)
X_meta_val = torch.tensor(X_meta_val, dtype=torch.float).to(device)
y_meta_val = torch.tensor(y_meta_val, dtype=torch.long).to(device)
X_meta_test = torch.tensor(X_meta_test, dtype=torch.float).to(device)
y_meta_test = torch.tensor(y_meta_test, dtype=torch.long).to(device)

all_preds, all_targets, all_outputs = [], [], []
train_losses, val_losses = [], []

best_val_f1 = 0
epochs_no_improve = 0
early_stop_threshold = 2

best_f1 = 0.0

# Training loop for the meta-model
epochs = 80

for epoch in range(epochs):
    meta_model.train()
    train_loss = 0
    for X_batch, y_batch in train_meta_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = meta_model(X_batch)
        loss = criterion(outputs, y_batch)
        l1_penalty = l1_regularization(meta_model, lambda_l1)
        loss += l1_penalty
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_losses.append(train_loss / len(train_meta_loader))

    meta_model.eval()
    val_loss = 0
    all_preds, all_targets, all_outputs = [], [], []
    with torch.no_grad():
        for X_batch, y_batch in val_meta_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = meta_model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(y_batch.cpu().numpy())
            all_outputs.extend(outputs.cpu().numpy())
    val_losses.append(val_loss / len(val_meta_loader))

    precision_0, recall_0, f1_0, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted', zero_division=0)
    accuracy = accuracy_score(all_targets, all_preds)

    output_str = f'Epoch {epoch+1}: Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Precision: {precision_0:.4f}, Recall: {recall_0:.4f}, F1: {f1_0:.4f}, Accuracy: {accuracy:.4f}\n'
    print(output_str)

    if f1_0 > best_f1:
        best_f1 = f1_0
        best_epoch = epoch
        best_model_state_dict = meta_model.state_dict()
        best_all_targets = all_targets
        best_all_preds = all_preds
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_threshold:
        print("Stopping early due to no improvement")
        break

meta_model.load_state_dict(best_model_state_dict)


## Inference on test set

In [ ]:
# Evaluate on the test set
meta_model.eval()
test_loss = 0
all_test_preds, all_test_targets = [], []
with torch.no_grad():
    for X_batch, y_batch in test_meta_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = meta_model(X_batch)
        loss = criterion(outputs, y_batch)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        all_test_preds.extend(predicted.cpu().numpy())
        all_test_targets.extend(y_batch.cpu().numpy())

test_loss /= len(test_meta_loader)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(all_test_targets, all_test_preds, average='weighted', zero_division=0)
accuracy_test = accuracy_score(all_test_targets, all_test_preds)

print(f'Test Loss: {test_loss:.4f}, Test Precision: {precision_test:.4f}, Test Recall: {recall_test:.4f}, Test F1: {f1_test:.4f}, Test Accuracy: {accuracy_test:.4f}')

### Conf-matrix and class-wise acc for meta-learner

In [27]:
conf_matrix = confusion_matrix(all_targets, all_preds)
# conf_matrix = confusion_matrix(all_preds, all_targets)
labels = ["background", "tackle-live", "tackle-replay"]
#labels = ["background", "tackle-live", "tackle-replay", "tackle-live-incomplete", "tackle-replay-incomplete"]


sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
# plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
#plt.savefig(f"{folder1}/meta/MetaModel_stretched.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Calculate accuracy per class
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

# Prepare accuracy data for writing to file
accuracy_data = "\n".join([f"Accuracy for class {i}: {class_accuracies[i]:.4f}" for i in range(len(class_accuracies))])
print(classification_report(all_targets, all_preds, target_names=labels))


# Print accuracy per class and write to a file
print(accuracy_data)  # Print to console

#torch.save(meta_model.state_dict(), f'{folder1}/meta/meta_model.pt')

# Define the filename
#accuracy_file_path = os.path.join(folder, "class_accuracies.txt")

# Write accuracies to a file in the specified folder
#with open(accuracy_file_path, 'w') as f:
#    f.write(f"Samples: {len(all_preds)}\n")  # Write the number of samples
#    f.write(accuracy_data)  # Write the accuracy data

## Roc curve

In [64]:
import numpy as np
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

# Class names and count
class_names = ['background', 'tackle-live', 'tackle-replay']
n_classes = len(class_names)

# Binarize the targets and predictions for ROC curve computation
test_targets_bin = label_binarize(targets, classes=[0, 1, 2])
test_predictions_bin = label_binarize(predictions, classes=[0, 1, 2])

# ROC curve and AUC for each class
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(test_targets_bin[:, i], test_predictions_bin[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves for each class
plt.figure(figsize=(8, 6))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'{class_names[i]} (AUC = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.grid(visible=True)
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC Curve')
plt.legend(loc='lower right')
plt.savefig("baseline-roc.pdf", format="pdf", bbox_inches="tight")
plt.show()

## Check model agreement

In [241]:
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def evaluate_model_agreement(meta_model, base_models, val_loader, device, criterion):
    meta_model.eval()
    agreement_counts = [0] * len(base_models)  # Agreement count for each base model
    total_predictions = 0  # Total predictions made

    all_preds = []
    all_targets = []

    with torch.no_grad():
        for X_batch, y_meta_val in val_loader:
            X_batch = X_batch.to(device)
            y_meta_val = y_meta_val.to(device)

            # Get predictions from each base model and the MetaModel
            base_probs = [torch.softmax(model(X_batch), dim=1) for model in base_models]
            base_predictions = [torch.max(probs, 1)[1] for probs in base_probs]

            # Concatenate the model outputs along feature dimension for MetaModel input
            meta_input = torch.cat(base_probs, dim=1)
            meta_outputs = meta_model(meta_input)
            meta_predictions = torch.max(meta_outputs, 1)[1]

            # Compare MetaModel predictions with each base model's predictions
            for i, base_preds in enumerate(base_predictions):
                agreement_counts[i] += (base_preds == meta_predictions).sum().item()

            total_predictions += y_meta_val.size(0)
            
            # Collect predictions for evaluation
            all_preds.extend(meta_predictions.cpu().numpy())
            all_targets.extend(y_meta_val.cpu().numpy())

            # Compute loss (optional)
            val_loss = criterion(meta_outputs, y_meta_val)

    # Calculate precision, recall, f1-score, and accuracy
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted', zero_division=0)
    accuracy = accuracy_score(all_targets, all_preds)

    # Calculate agreement percentages
    agreement_percentages = [count / total_predictions * 100 for count in agreement_counts]

    print(f"Loss: {val_loss.item()}")
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Accuracy: {accuracy:.4f}')
    return agreement_percentages

# Usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#meta_model = best_meta_model  # Your loaded MetaModel
base_models = [model0, model1, model2]  # List of your base models
criterion = torch.nn.CrossEntropyLoss()  # Your loss function

agreement_percentages = evaluate_model_agreement(meta_model, base_models, val_loader, device, criterion)
for i, pct in enumerate(agreement_percentages):
    print(f"Model {i} Agreement Percentage: {pct:.2f}%")

    
print(classification_report(all_targets, all_preds, target_names=labels))
    

## Inference

In [28]:
all_preds, all_targets, all_outputs = [], [], []
import time

batch_times = []

with torch.no_grad():
    start_time = time.time()
    
    for batch_idx, (X_batch, y_meta_val) in enumerate(test_loader):
        X_batch = X_batch.to(device)
        # Store the probabilities, not the class predictions
        probs0 = torch.softmax(model0(X_batch), dim=1)
        probs1 = torch.softmax(model1(X_batch), dim=1)
        probs2 = torch.softmax(model2(X_batch), dim=1)
        
        # Concatenate the model outputs along feature dimension
        model_output = torch.cat((probs0, probs1, probs2), dim=1)
        
        val_outputs = meta_model(model_output)
        

        val_loss = criterion(val_outputs, y_meta_val)

        _, predicted = torch.max(val_outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(y_meta_val.cpu().numpy())
        all_outputs.extend(val_outputs.cpu().numpy())
        batch_time = time.time() - start_time
        batch_times.append(batch_time)
        print(f"Batch {batch_idx + 1}: Time = {batch_time:.7f} seconds")

    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted', zero_division=0)
    accuracy = accuracy_score(all_targets, all_preds)
    
    print(f"Loss: {loss.item()}, Val Loss: {val_loss.item()}")
        
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Accuracy: {accuracy:.4f}')

average_batch_time = sum(batch_times) / len(batch_times)

print(f"Average Batch Time: {average_batch_time:.7f} seconds")

#torch.save(meta_model.state_dict(), '/home/evan/D1/project/code/meta_model/meta_model_3')